In [1]:
import copy
import numpy as np
from persor import BVHparser
from scipy.spatial.transform import Rotation

In [2]:
def get_joint_coords(skeleton, joint, coords):
    offset = skeleton[joint]["offset"]
    parent_joint = skeleton[joint]["joint"]
    child_joints = skeleton[joint]["children"]

    if parent_joint == None:
        return

    parent_coord = coords[parent_joint]["coord"]

    current_coord = [
        parent_coord[0] + offset[0],
        parent_coord[1] + offset[1],
        parent_coord[2] + offset[2],
    ]
    coords[joint] = {"coord": current_coord, "parent": parent_joint}

    for child in child_joints:
        get_joint_coords(skeleton, child, coords)


def skelton2coords(skeleton):
    coords = {"root": {"coord": skeleton["root"]["offset"], "parent": None}}

    child_joints = skeleton["root"]["children"]
    for child in child_joints:
        get_joint_coords(skeleton, child, coords)

    get_joint_coords(skeleton, "root", coords)

    return coords


def get_rotate_vec(frame, path):
    x_rotate = sum([frame[f"{joint}_Xrotation"] for joint in path])
    y_rotate = sum([frame[f"{joint}_Yrotation"] for joint in path])
    z_rotate = sum([frame[f"{joint}_Zrotation"] for joint in path])
    x_position = frame[f"{path[0]}_Xposition"]
    y_position = frame[f"{path[0]}_Yposition"]
    z_position = frame[f"{path[0]}_Zposition"]

    R_x = Rotation.from_euler("x", x_rotate, degrees=True).as_matrix()
    R_y = Rotation.from_euler("y", y_rotate, degrees=True).as_matrix()
    R_z = Rotation.from_euler("z", z_rotate, degrees=True).as_matrix()

    return R_y @ R_x @ R_z @ np.array([x_position, y_position, z_position])


def get_path2root(skeleton, joint):
    path = []
    while True:
        path.append(joint)
        joint = skeleton[joint]["joint"]
        if joint == None:
            break
    return path


def rotated_skeleton(skeleton, frame):
    skeleton_copy = copy.deepcopy(skeleton)

    for joint in skeleton.keys():
        if joint.startswith("_"):
            continue

        path = get_path2root(skeleton, joint)
        skeleton_copy[joint]["offset"] = get_rotate_vec(frame, path)

    return skeleton_copy


def convert2coordDict(df, skeleton):
    df = df.copy()
    data = []
    for i in range(len(df)):
        skeleton_rotated = rotated_skeleton(skeleton, df.iloc[i])
        coords = skelton2coords(skeleton_rotated)
        data.append(coords)

    return np.array(data)


def coord_dict2coords_array(coords_dict):
    return np.array(
        [[data["coord"] for joint, data in coords.items()] for coords in coords_dict]
    )


def get_splited_data(bvhp, window_size, stride, start=0, label=0):
    df = bvhp.get_motion_df()
    skeleton = bvhp.get_skeleton()
    coord_data = coord_dict2coords_array(convert2coordDict(df, skeleton))

    data = np.array(
        [
            coord_data[i : i + window_size]
            for i in range(start, len(df) - window_size, stride)
        ]
    ).transpose(0, 3, 1, 2)
    labels = [label] * len(data)

    return data, labels

## 訓練・テスト用データを生成するよ

1. 体を曲げる運動
2. 両手を上げ下げ
3. 体を捻る
4. ジャンプ
5. 何もしない

In [3]:
# データの取得
window_size = 200
data1, label1 = get_splited_data(BVHparser("./data/bvh/体を曲げる運動.bvh"), window_size, 300, 270, 0)
data2, label2 = get_splited_data(BVHparser("./data/bvh/両手を上げ下げ.bvh"), window_size, 300, 270, 1)
data3, label3 = get_splited_data(BVHparser("./data/bvh/体を捻る.bvh"), window_size, 300, 270, 2)
data4, label4 = get_splited_data(BVHparser("./data/bvh/ジャンプ.bvh"), window_size, 300, 280, 3)
data5, label5 = get_splited_data(BVHparser("./data/bvh/何もしない.bvh"), window_size, 300, 270, 4)

# データを結合
data = np.concatenate([data1, data2, data3, data4, data5])
label = np.concatenate([label1, label2, label3, label4, label5])

# data, labelをシャッフル
p = np.random.permutation(len(data))
data = data[p]
label = label[p]

# 訓練データとテストデータに分割
train_data = data[: int(len(data) * 0.8)]
train_label = label[: int(len(data) * 0.8)]
test_data = data[int(len(data) * 0.8) :]
test_label = label[int(len(data) * 0.8) :]

np.save("data/train_data.npy", train_data)
np.save("data/train_label.npy", train_label)
np.save("data/test_data.npy", test_data)
np.save("data/test_label.npy", test_label)

print(set(train_label))

{np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)}


In [4]:
# data の shape
print(data.shape)

(125, 3, 200, 27)
